To run this locally, [install Ploomber](https://docs.ploomber.io/en/latest/get-started/quick-start.html) and execute: `ploomber examples -n cookbook/python-load`

Found an issue? [Let us know.](https://github.com/ploomber/projects/issues/new?title=cookbook/python-load%20issue)

Questions? [Ask us on Slack.](https://ploomber.io/community/)


# Loading pipeline.yaml in Python

<!-- start description -->
Load pipeline.yaml file in a Python session to customize initialization.
<!-- end description -->

## Loading the pipeline

Create a function that loads it and returns the DAG object:

```python
# Content of pipeline.py
from ploomber.constants import TaskStatus
from ploomber.spec import DAGSpec
from ploomber import with_env


@with_env('env.yaml')
# NOTE: you may add extra arguments to the function, they'll show up in the cli
def make(env):
    """Simplest factory function

    Examples
    --------
    Execute in the terminal:
        ploomber build -e pipeline.make
    """
    dag = DAGSpec('pipeline.yaml', env=dict(env)).to_dag()
    # NOTE: a factory function must return a DAG (do not call any methods)
    return dag


@with_env('env.yaml')
def delete_tasks(env):
    """Factory function that deletes tasks based on an input parameter

    Examples
    --------
    Execute in the terminal:
        ploomber build -e pipeline.delete_tasks --env--delete_tasks true
    """
    dag = DAGSpec('pipeline.yaml', env=dict(env)).to_dag()

    if env.delete_tasks:
        del dag['create_another']
        del dag['create_final']

    return dag


@with_env('env.yaml')
def ignore_task_status(env):
    """Factory function that overwrites a task's status and skips it

    Examples
    --------
    Execute in the terminal:
        ploomber task create_file --force
        ploomber build -e pipeline.ignore_task_status --force
    """
    dag = DAGSpec('pipeline.yaml', env=dict(env)).to_dag()
    dag.render()

    # NOTE: this example uses private APIs, and they may not work in the
    # future. Keep an eye on: https://github.com/ploomber/ploomber/issues/681
    # for updates

    # never execute the "create_file" tasl
    dag['create_file']._exec_status = TaskStatus.Skipped
    # this is necessary to prevent ploomber from overwriting the status
    dag._params.cache_rendered_status = True
    return dag

```

*Note:* If your pipeline isn't using an `env.yaml` file, remove the decorator and the `env` argument in the function.

## Command-line interface

The CLI will work just as if you were using the `pipeline.yaml` file directly, pass the dotted path to the function in the `--entry-point/-e` argument:

In [1]:
%%sh
ploomber status -e pipeline.make

Loading pipeline...
name         Last run    Outdated?    Product      Doc        Location
                                                   (short)
-----------  ----------  -----------  -----------  ---------  -----------
create_file  Has not     Source code  File('produ  None       /Users/Edu/
             been run                 cts/number/             dev/project
                                      some_file.t             s-ploomber/
                                      xt')                    cookbook/py
                                                              thon-load/t
                                                              asks.py:4
create_anot  Has not     Source code  File('produ  None       /Users/Edu/
her          been run    & Upstream   cts/number/             dev/project
                                      another_fil             s-ploomber/
                                      e.txt')                 cookbook/py
                                      

/Users/Edu/miniconda3/envs/projects/lib/python3.9/site-packages/ploomber/spec/dagspec.py:353: UserWarning: The following placeholders are declared in the environment but unused in the spec: {'delete_tasks', 'user', 'cwd', 'git_hash', 'here', 'root', 'now', 'git'}
  warnings.warn('The following placeholders are declared in the '
100%|██████████| 3/3 [00:00<00:00, 3601.29it/s]


To build the pipeline:

In [2]:
%%sh
ploomber build -e pipeline.make

Loading pipeline...
name            Ran?      Elapsed (s)    Percentage
--------------  ------  -------------  ------------
create_file     True         0.000568       38.3266
create_another  True         0.000443       29.892
create_final    True         0.000471       31.7814


/Users/Edu/miniconda3/envs/projects/lib/python3.9/site-packages/ploomber/spec/dagspec.py:353: UserWarning: The following placeholders are declared in the environment but unused in the spec: {'root', 'now', 'user', 'delete_tasks', 'git_hash', 'cwd', 'here', 'git'}
  warnings.warn('The following placeholders are declared in the '
/Users/Edu/miniconda3/envs/projects/lib/python3.9/site-packages/ploomber/cli/cloud.py:164: UserWarning: Issue: No Valid API_Key
  warnings.warn(content)
Building task 'create_final': 100%|██████████| 3/3 [00:08<00:00,  2.70s/it]  
/Users/Edu/miniconda3/envs/projects/lib/python3.9/site-packages/ploomber/cli/cloud.py:164: UserWarning: Issue: No Valid API_Key
  warnings.warn(content)


Note that you can modify values set in the `env.yaml`, to see how, pass ``--help``:

In [3]:
%%sh
ploomber build -e pipeline.make --help

Loading pipeline...
usage: ploomber build [-h] [--log LOG] [--log-file LOG_FILE]
                      [--entry-point ENTRY_POINT] [--force] [--skip-upstream]
                      [--partially PARTIALLY] [--debug]
                      [--env--run_number ENV__RUN_NUMBER]
                      [--env--delete_tasks ENV__DELETE_TASKS]

Build pipeline. Simplest factory function

optional arguments:
  -h, --help            show this help message and exit
  --log LOG, -l LOG     Enables logging to stdout at the specified level
  --log-file LOG_FILE, -F LOG_FILE
                        Enables logging to the given file
  --entry-point ENTRY_POINT, -e ENTRY_POINT
                        Entry point, defaults to pipeline.yaml
  --force, -f           Force execution by ignoring status
  --skip-upstream, -su  Skip building upstream dependencies. Only applicable
                        when using --partially
  --partially PARTIALLY, -p PARTIALLY
                        Build a pipeline partially 

*Note:* Check out the other functions in `pipeline.yaml` for more examples.